In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

energy = pd.read_excel("/Users/Farida/Desktop/KAMRAN/kaggle/energy.xlsx")

In [2]:
energy.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [3]:
# Delete Y2 from the dataframe
y1data = energy.drop('Y2', axis=1)
# Delete Y1 from the dataframe
y2data = energy.drop('Y1', axis=1)

In [4]:
y1data[:2]

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55


In [5]:
y2data[:2]

,X1,X2,X3,X4,X5,X6,X7,X8,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,21.33


In [6]:
# Make inputs dataframe
inputs = y1data.drop('Y1', axis=1)
# Make output Y1 list
output1 = y1data['Y1']
# Make output Y2 list
output2 = y2data['Y2']

In [7]:
inputs[:2]

,X1,X2,X3,X4,X5,X6,X7,X8
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0


In [8]:
output1[:2]

0    15.55
1    15.55
Name: Y1, dtype: float64

In [9]:
output2[:2]

0    21.33
1    21.33
Name: Y2, dtype: float64

In [10]:
# Split the data randomly to test and train for input and output
X_train, X_test, y_train, y_test = train_test_split(inputs, output1, test_size=.20, random_state=123)

In [11]:
# Making a basic Linear Regression model for output Y1
model1 = linear_model.LinearRegression()
model1.fit(X_train,y_train)
Y_hat1 = model1.predict(X_test)
accuracy1 = model1.score(X_test,y_test)
print(accuracy1)

0.9031821255806276


/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [12]:
# Make a client_data list that contains all the lists of all X values row wise
client_data = []
for index, row in X_test.iterrows():
    x = []
    x.append(row['X1'])
    x.append(row['X2'])
    x.append(row['X3'])
    x.append(row['X4'])
    x.append(row['X5'])
    x.append(row['X6'])
    x.append(row['X7'])
    x.append(row['X8'])
    client_data.append(x)

In [13]:
# Making a list of Y1 values
testdatay1 = []
for i in y_test:
    testdatay1.append(i)

In [14]:
resultY1 = []
for i, price in enumerate(model1.predict(client_data)):
    x = testdatay1[i]
    minus = x - price
    percent = minus*x/100
    percent = round(percent, 2)
    price = round(price, 2)
    resultY1.append({'predictY1': price, 'trueY1': x, 'errorY1': percent})

resY1 = pd.DataFrame(resultY1)

In [15]:
resY1[:2]

,errorY1,predictY1,trueY1
0,-0.03,13.05,12.85
1,-0.22,29.83,29.07


In [16]:
# Split the data randomly to test and train for input and output
X_train, X_test, y_train, y_test = train_test_split(inputs, output2, test_size=.20, random_state=123)

In [17]:
# Making a basic Linear Regression model for output Y2
model2 = linear_model.LinearRegression()
model2.fit(X_train,y_train)
Y_hat2 = model2.predict(X_test)
accuracy2 = model2.score(X_test,y_test)
print(accuracy2)

0.880053669252084


In [18]:
# Making a list of Y2 values
testdatay2 = []
for i in y_test:
    testdatay2.append(i)

In [19]:
# Making a dataframe of Y2 predicted, Y2 true and error percent between them
resultY2 = []
for i, price in enumerate(model2.predict(client_data)):
    x = testdatay2[i]
    difference = x - price
    percent = difference*x/100
    percent = round(percent, 2)
    price = round(price, 2)
    resultY2.append({'predictY2': price, 'trueY2': x, 'errorY2': percent})

resY2 = pd.DataFrame(resultY2)

In [20]:
# Combining both result dataframes
result = pd.concat([resY1, resY2], axis=1)
result[:3]

,errorY1,predictY1,trueY1,errorY2,predictY2,trueY2
0,-0.03,13.05,12.85,-0.12,15.23,14.37
1,-0.22,29.83,29.07,-0.75,32.34,29.82
2,-0.12,8.91,7.18,-0.07,12.98,12.40


In [21]:
# Calculating the average error between two errors for each row, saving all in a list
averr = []
for index, row in result.iterrows():
    x = row['errorY1']
    y = row['errorY2']
    z = (abs(x) + abs(y)) / 2
    z = round(z, 3)
    averr.append(z)

In [22]:
# Turning the above list into a dataframe
average = pd.DataFrame(averr)

In [23]:
# Chaning the name of the column
average.rename(columns={0: 'averageError'}, inplace=True)

In [24]:
# Concatenating the average error dataframe with both combined result dataframes
fulldatabase = pd.concat([result, average], axis=1)

In [25]:
fulldatabase[:10]

,errorY1,predictY1,trueY1,errorY2,predictY2,trueY2,averageError
0,-0.03,13.05,12.85,-0.12,15.23,14.37,0.075
1,-0.22,29.83,29.07,-0.75,32.34,29.82,0.485
2,-0.12,8.91,7.18,-0.07,12.98,12.40,0.095
3,1.82,30.30,35.45,3.78,32.83,41.86,2.800
4,0.03,28.49,28.61,0.11,29.84,30.20,0.070
5,-0.22,40.42,39.86,-0.84,40.37,38.18,0.530
6,0.04,11.45,11.80,-0.00,14.52,14.49,0.020
7,-0.24,14.83,12.95,-0.16,16.93,15.95,0.200
8,0.47,12.28,15.36,0.82,15.02,19.29,0.645
9,-0.22,33.91,33.24,-0.41,35.70,34.50,0.315


In [26]:
average_accuracy = (accuracy1 + accuracy2) / 2 * 100
print('Prediction accuracy ~',round(average_accuracy, 2), '%')

Prediction accuracy ~ 89.16 %
